# Phenopacket store statistics

This notebook performs quality assessment and calculate descriptive statistics about a phenopacket-store release. 

The input file is the zip file that is or will be added to each release.

In [1]:
from ppktstore.stats import PPKtStoreStats
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
stats = PPKtStoreStats(input_zipfile="all_phenopackets.zip")

In [3]:
df = stats.get_df()
df.head(2)

disease  \
0  Neurodevelopmental disorder with intracranial hemorrhage, seizures, and spasticity   
1  Neurodevelopmental disorder with intracranial hemorrhage, seizures, and spasticity   

    disease_id            patient_id  gene              allele_1 allele_2  \
0  OMIM:620371  Individual KCHYD24-1  ESAM  NM_138961.3:c.605T>G            
1  OMIM:620371          Individual 1  ESAM  NM_138961.3:c.115del            

            PMID  
0  PMID:36996813  
1  PMID:36996813

In [4]:
df.columns

Index(['disease', 'disease_id', 'patient_id', 'gene', 'allele_1', 'allele_2',
       'PMID'],
      dtype='object')

In [5]:
stats_d = stats.get_descriptive_stats()
items = list()
for k,v in stats_d.items():
    items.append({"item": k, "value": v})
df = pd.DataFrame(items)
df.head(30)

item                           value
0                              phenopackets                            4343
1                                  diseases                             395
2                                     genes                             357
3                                   alleles                            2289
4                                     PMIDs                             661
5             individuals per disease (max)                             464
6             individuals per disease (min)                               1
7            individuals per disease (mean)                       10.994937
8          individuals per disease (median)                             1.0
9           individuals per disease (n>=10)                              92
10          individuals per disease (n>=20)                              47
11          individuals per disease (n>=50)                              16
12         individuals per disease (n>=100)                               5
13   genes associated with a single disease                               0
14       genes associated with two diseases                               0
15  genes associated with multiple diseases                               2
16     gene with maximum number of diseases  1 with 392 associated diseases
17               individuals per gene (max)                             464
18               individuals per gene (min)                               1
19              individuals per gene (mean)                       12.165266
20            individuals per gene (median)                             1.0
21             individuals per gene (n>=10)                              76
22             individuals per gene (n>=20)                              41
23             individuals per gene (n>=50)                              18
24            individuals per gene (n>=100)                               7

In [6]:
#df.groupby("disease_id").count()
df["unique_id"] = df["patient_id"] + df["PMID"]
df.head()

KeyError: 'patient_id'

In [ ]:
grouped_by_disease = df.groupby("disease_id").count()
grouped_by_disease.head()


disease  patient_id  gene  allele_1  allele_2  PMID  unique_id  \
disease_id                                                                    
OMIM:101200        2           2     2         2         2     2          2   
OMIM:102370       14          14    14        14        14    14         14   
OMIM:102500        1           1     1         1         1     1          1   
OMIM:102700        1           1     1         1         1     1          1   
OMIM:103500        1           1     1         1         1     1          1   

             disease_count  
disease_id                  
OMIM:101200              0  
OMIM:102370              0  
OMIM:102500              0  
OMIM:102700              0  
OMIM:103500              0

In [ ]:
counts_per_disease = stats.get_counts_per_disease()
counts_per_disease.head(2)

disease_id
OMIM:612164    464
OMIM:613721    343
Name: count, dtype: int64

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_with_max(
        disease_counts: pd.Series,
        max_val: int,
        ax=None,
        ):
    assert isinstance(max_val, int) and max_val > 0, '`max_val` must be a positive `int`'

    if ax is None:
        _, ax = plt.subplots(figsize=(7, 4), dpi=300)

    bins = list(range(max_val + 1)) + [disease_counts.max() + 1]
    labels = list(map(str, bins[:-2])) + [f'  ≥{max_val}']
    cats = pd.cut(
        x=disease_counts,
        bins=bins,
        labels=labels,
        right=False,
    )
    cat_count = cats.value_counts().sort_index()
    ax.bar(x=cat_count.index, height=cat_count.values)
    ax.set(
        xlabel='Number of cases',
        ylabel='Disease count',
    )
    ax.grid(axis='y')


In [ ]:
plot_with_max(
    disease_counts=counts_per_disease,
    max_val=25,
)
plt.savefig('counts_per_disease.png')

# Check all disease identifiers
They should all be CURIEs with the prefixes OMIM or MONDO. There should be no whitespace between the colon and the suffix.

In [ ]:
df = stats.check_disease_id()

No problems found.


# Count diseases in cohort
In phenopacket-store, each of the folders underneath the "notebooks" folder is counted as a cohort. The following function can be used to 
display the count of diseases in a specific cohort. The intended use case is that in time we would like to move the LIRICAL phenopackets into gene-specific cohorts.

In [ ]:
input_zipfile = "all_phenopackets.zip"
target = "LIRICAL"
df = stats.count_diseases_in_cohort(input_zipfile=input_zipfile, cohort=target)
df.head(50)

label  \
disease_id                                                                                                                     
OMIM:616900                                   Hypotonia, infantile, with psychomotor retardation and characteristic facies 3   
OMIM:616907                                                                       Spastic paraplegia 76, autosomal recessive   
OMIM:159550                                                                                     Ataxia-pancytopenia syndrome   
OMIM:617425                                                   Immunoskeletal dysplasia with neurodevelopmental abnormalities   
OMIM:617560                                       Spastic ataxia 8, autosomal recessive, with hypomyelinating leukodystrophy   
OMIM:616829                                                                   Congenital disorder of glycosylation, type IIp   
OMIM:616878  Metabolic encephalomyopathic crises, recurrent, with rhabdomyolysis, cardiac arrhythmias, and neurodegeneration   
OMIM:154700                                                                                             MARFAN SYNDROME; MFS   
OMIM:617516                                                                                      Stankiewicz-Isidor syndrome   
OMIM:616840                                                           Parkinson disease 23, autosomal recessive, early onset   
OMIM:250410                                                          Retinitis pigmentosa with or without skeletal anomalies   
OMIM:613454                                                                                RETT SYNDROME, CONGENITAL VARIANT   
OMIM:231200                                                                                    BERNARD-SOULIER SYNDROME; BSS   
OMIM:216400                                                                                         COCKAYNE SYNDROME A; CSA   
OMIM:613684                                                                               RUBINSTEIN-TAYBI SYNDROME 2; RSTS2   
OMIM:122470                                                                              CORNELIA DE LANGE SYNDROME 1; CDLS1   
OMIM:226700                                                                  EPIDERMOLYSIS BULLOSA, JUNCTIONAL, HERLITZ TYPE   
OMIM:214800                                                                                                  CHARGE SYNDROME   
OMIM:133540                                                                                         COCKAYNE SYNDROME B; CSB   
OMIM:191100                                                                                       TUBEROUS SCLEROSIS 1; TSC1   
OMIM:600373                                                                                                   CODAS SYNDROME   
OMIM:177170                                                                                      PSEUDOACHONDROPLASIA; PSACH   
OMIM:236200                                                     HOMOCYSTINURIA DUE TO CYSTATHIONINE BETA-SYNTHASE DEFICIENCY   
OMIM:616974                                                                 Combined oxidative phosphorylation deficiency 30   
OMIM:182290                                                                                      SMITH-MAGENIS SYNDROME; SMS   
OMIM:617433                                                                                          Retinitis pigmentosa 78   
OMIM:611431                                                                                                  LEGIUS SYNDROME   
OMIM:601977                                                                                        THROMBOCYTHEMIA 2; THCYT2   
OMIM:103500                                                                                                   TIETZ SYNDROME   
OMIM:131100                                                                       MULTIPLE ENDOCRINE NEOPLASIA, TYPE I; MEN1   
OMIM:266265            

# Check for duplicates
In some cases, duplicate phenopackets were inadvertently added to some of the initial cohorts at earlier stages of the project. This function lists phenopacket identifers arranged according to variant and allows us to more easily check for this kind of error.

In [ ]:
df = stats.show_possible_duplicates_by_variant(input_zipfile, "ERI1")

No candidate duplicates found for ERI1


# Quality control
The following commands perform several consistency checks.

In [ ]:
df = stats.find_phenopackets_with_no_variants(input_zipfile=input_zipfile)

All phenopackets had at least one variant


In [ ]:
stats.show_possible_duplicates_by_variant(input_zipfile, "ARPC5")

No candidate duplicates found for ARPC5


Empty DataFrame
Columns: []
Index: []

In [ ]:
stats.find_phenopackets_with_no_disease(input_zipfile)

All phenopackets had a disease diagnosis


Empty DataFrame
Columns: []
Index: []